In [1]:
import pandas as pd

pd_dat = "cleaned_industrial_data.csv"

data = pd.read_csv(pd_dat)

print(data.head())


  PERMIT_TYPE  APP_ID         WDID      STATUS NOI_PROCESSED_DATE  \
0  Industrial  153338  5A327021N05  Terminated         2006-05-14   
1  Industrial  178202  1 08I001442  Terminated         1992-03-27   
2  Industrial  178203  1 08I004046      Active         1992-04-03   
3  Industrial  178204  1 08I015378      Active         1999-09-20   
4  Industrial  178205  1 08I017295      Active         2002-06-11   

  REGION_BOARD     COUNTY                               OPERATOR_NAME  \
0           5R     Plumas                      Union Pacific Railroad   
1            1  Del Norte  Del Norte Solid Waste Management Authority   
2            1  Del Norte                   Hambro Forest Product Inc   
3            1  Del Norte                                 Bruce Gates   
4            1  Del Norte              Green Diamond Resource Company   

                    FACILITY_NAME           FACILITY_ADDRESS  ...  \
0  Union Pacific Portola Railyard                     1 Park  ...   
1       

In [2]:

data['FACILITY_ZIP'] = data['FACILITY_ZIP'].astype(str).str.zfill(5)


data = data.dropna(subset=['PERMIT_TYPE', 'FACILITY_NAME', 'STATUS'])


data['FACILITY_LATITUDE'] = data['FACILITY_LATITUDE'].astype(float)
data['FACILITY_LONGITUDE'] = data['FACILITY_LONGITUDE'].astype(float)


print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53330 entries, 0 to 53329
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   PERMIT_TYPE                  53330 non-null  object 
 1   APP_ID                       53330 non-null  int64  
 2   WDID                         53330 non-null  object 
 3   STATUS                       53330 non-null  object 
 4   NOI_PROCESSED_DATE           53330 non-null  object 
 5   REGION_BOARD                 53330 non-null  object 
 6   COUNTY                       53330 non-null  object 
 7   OPERATOR_NAME                53330 non-null  object 
 8   FACILITY_NAME                53330 non-null  object 
 9   FACILITY_ADDRESS             53330 non-null  object 
 10  FACILITY_CITY                53330 non-null  object 
 11  FACILITY_STATE               53330 non-null  object 
 12  FACILITY_ZIP                 53330 non-null  object 
 13  FACILITY_LATITUD

In [3]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///environmental_compliance.db')

data.to_sql('FACILITY', con=engine, if_exists='replace', index=False)

print("Data successfully loaded into the database.")


Data successfully loaded into the database.


In [4]:
import smtplib
from email.mime.text import MIMEText
import pandas as pd

data = pd.read_sql("SELECT * FROM FACILITY", con=engine)


inactive_facilities = data[data['STATUS'] == 'Inactive']


def send_email(facilities):
    recipient = "regulatory_team@example.com"
    subject = "Inactive Permits Alert"
    body = f"Facilities with inactive permits:\n\n{facilities[['FACILITY_NAME', 'COUNTY']].to_string(index=False)}"
    
    msg = MIMEText(body)
    msg['From'] = "alerts@example.com"
    msg['To'] = recipient
    msg['Subject'] = subject


    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()
        server.login("your_email@gmail.com", "your_password")
        server.sendmail(msg['From'], [msg['To']], msg.as_string())

send_email(inactive_facilities)


SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d75a77b69052e-46a3e676f26sm25893751cf.19 - gsmtp')

In [10]:
!pip install folium

In [13]:
import folium

# Filter data for active facilities
active_facilities = data[data['STATUS'] == 'Active']

# Create a base map
map = folium.Map(location=[36.7783, -119.4179], zoom_start=6)  # California coordinates

# Add markers for facilities
for _, row in active_facilities.iterrows():
    folium.Marker(
        location=[row['FACILITY_LATITUDE'], row['FACILITY_LONGITUDE']],
        popup=f"{row['FACILITY_NAME']} ({row['COUNTY']})",
        icon=folium.Icon(color='green')
    ).add_to(map)

# Save the map to an HTML file
map.save("active_facilities_map.html")

print(map.save)


<bound method Element.save of <folium.folium.Map object at 0x000002B2B6E7A7C0>>


In [12]:
import matplotlib.pyplot as plt

# Load discharge data for a specific facility
facility_name = "Facility X"
facility_data = data[data['FACILITY_NAME'] == facility_name]

# Plot discharge trends
plt.figure(figsize=(10, 6))
plt.plot(facility_data['DISCHARGE_DATE'], facility_data['DISCHARGE_VOLUME'], marker='o')

# Customize the plot
plt.title(f"Discharge Trends for {facility_name}")
plt.xlabel("Date")
plt.ylabel("Discharge Volume")
plt.grid(True)
plt.tight_layout()

# Save or show the plot
plt.savefig("discharge_trends.png")
print("Discharge trend plot saved as discharge_trends.png.")

print(plt.savefig)


KeyError: 'DISCHARGE_DATE'

<Figure size 720x432 with 0 Axes>

In [14]:
# Filter data for a specific county
county_data = data[data['COUNTY'] == 'Los Angeles']

# Export filtered data
county_data.to_csv('los_angeles_facilities.csv', index=False)

print(county_data.to_csv)


<bound method NDFrame.to_csv of       PERMIT_TYPE  APP_ID           WDID          STATUS NOI_PROCESSED_DATE  \
1116   Industrial  181814    2 07I002637      Terminated         1992-04-01   
2937   Industrial  188525    4 19I000008      Terminated         1992-01-26   
2938   Industrial  188526    4 19I000010      Terminated         1992-01-26   
2939   Industrial  188526  4 19Nec012103          Active         1992-01-26   
2940   Industrial  188527    4 19I000023      Terminated         1992-02-07   
...           ...     ...            ...             ...                ...   
53314  Industrial  578745  4 19Nec013951          Active         2024-11-04   
53317  Industrial  578761  4 19Nec013947          Active         2024-11-04   
53321  Industrial  578834  4 19Nna003068  Nona Submitted         2024-11-06   
53323  Industrial  578850  4 19Nec013948          Active         2024-11-04   
53327  Industrial  578913  4 19Nec013961          Active         2024-11-07   

      REGION_BOARD 